In [28]:
run Networks.ipynb

In [29]:
import networkx as nx
import time
from copy import copy

In [30]:
class Node:
	def __init__(self, x):
		self.ID = x
		self.paths = []
		self.isVisited = False
		self.hopCount = -1
		self.isReserved = False
	def resetNode(self):
		self.paths = []
		self.isVisited = []
		self.hopCount = -1

In [31]:
def addNeighbors(G, n, d):
    for neighbor in G.neighbors(n):
        if (not G.allNodes[neighbor].isReserved) and (G.allNodes[n].hopCount < G.limit):
            if (not G.allNodes[neighbor].isVisited) or (G.allNodes[neighbor].hopCount > G.allNodes[n].hopCount):
                G.allNodes[neighbor].isVisited = True
                G.allNodes[neighbor].hopCount = G.allNodes[n].hopCount + 1
                G.allNodes[neighbor].paths = [sub + [neighbor] for sub in G.allNodes[n].paths]
                if (neighbor != d) and (neighbor not in G.queue):
                    G.queue.append(neighbor)
                if (neighbor == d):
                    G.limit = G.allNodes[neighbor].hopCount

In [32]:
def is_disjoint(path1, path2):
    for i in range(1, len(path1) - 1):
        if path1[i] == path2[i]:
            return False
    return True

In [33]:
def filterDisjointPaths(current_index, current_paths, remaining_paths):
    if current_index == len(remaining_paths):
        return current_paths

    path = remaining_paths[current_index]
    disjoint = True
    for existing_path in current_paths:
        if not is_disjoint(path, existing_path):
            disjoint = False
            break

    with_current = current_paths.copy()
    if disjoint:
        with_current.append(path)

    without_current = filterDisjointPaths(current_index + 1, current_paths, remaining_paths)
    with_current = filterDisjointPaths(current_index + 1, with_current, remaining_paths)

    return with_current if len(with_current) > len(without_current) else without_current

In [34]:
def discoverPaths(G, source, destination):
    G_tmp = copy(G)
    G_tmp.queue.append(source)
    G_tmp.allNodes[source].hopCount = 0
    G_tmp.allNodes[source].isVisited = True
    G_tmp.allNodes[source].paths.append([source])
    while G_tmp.queue:
        activeNode = G_tmp.queue[0]
        G_tmp.allNodes[activeNode].paths = filterDisjointPaths(0, [], G_tmp.allNodes[activeNode].paths)
        addNeighbors(G_tmp, activeNode, destination)
        del G_tmp.queue[0]
    if G_tmp.allNodes[destination].isVisited:
        G.disjointPaths = G.disjointPaths + filterDisjointPaths(0, [], G_tmp.allNodes[destination].paths)
        for path in G.disjointPaths:
            for node in path:
                if node != destination:
                    G.allNodes[node].isReserved = True

In [35]:
def beginDiscovery(G, source, destination):
    if G.has_edge(source,destination):
        G.remove_edge(source,destination)
        G.disjointPaths.append([source,destination])
    while (len(G.disjointPaths) < G.degree(source)):
        discoverPaths(G, source, destination)
    return G.disjointPaths

In [42]:
# Mesh Networks: mesh
# Torus Networks: torus
# Gaussian Networks: gaussian | Gaussian8Plus9 | Gaussian8Plus11 | Gaussian9Plus9 | Gaussian8Plus10 | Gaussian7Plus13
# EJ Networks: EJ8Plus9 | EJ7Plus10 | EJ5Plus12 | EJ9Plus9 | EJ5Plus13
# Cube-Connected Cycles Networks: CCC4D
# Hypercube Networks: HC5D
testNetwork = CCC4D
G = nx.Graph()
G.add_nodes_from(testNetwork.Nodes)
G.add_edges_from(testNetwork.Edges)
G.allNodes = {}
G.reservedNodes = []
G.disjointPaths = []
G.queue = []
G.limit = len(testNetwork.Nodes)
for node in testNetwork.Nodes:
    G.allNodes[node]=Node(node)
    
start = time.time()
beginDiscovery(G, '0,0','7,2')
end = time.time()

G.disjointPaths

[['0,0', '1,0', '1,1', '3,1', '3,2', '7,2'],
 ['0,0', '1,0', '1,1', '3,1', '3,2', '7,2'],
 ['0,0', '1,0', '1,1', '3,1', '3,2', '7,2']]